Scripts to plot the MCMC-simulated spectra against DSN observations, visualize each rotational transition, and extract the corresponding model frequencies and intensities for spectral line stacking analysis. Bounds for HC<sub>5</sub>N and HC<sub>7</sub>N in Cha-C2 are included, and others can be added following the same format.

In [ ]:
import numpy as np
import pandas as pd
import sys
import matplotlib.pyplot as plt
from contextlib import contextmanager

from spectral_simulator.classes import *
from spectral_simulator.constants import *
from spectral_simulator.functions import *
from MCMC_scripts.Cha1_DSS43 import *

In [ ]:
with suppress_output():
    BASE_DIR = os.getcwd()

    config = {
        'mol_name': 'hc7n_hfs',
        'fit_folder': os.path.join(BASE_DIR, 'DSN_fit_results'),
        'cat_folder': os.path.join(BASE_DIR, 'CDMS_catalog'),
        'data_path': os.path.join(BASE_DIR, 'DSN_data', 'cha_c2_hc5n_example.npy'),
        'block_interlopers': True,
        'nruns': 1000,
        'restart': False,
        'prior_path': os.path.join(BASE_DIR, 'DSN_fit_results', 'hc5n_hfs', 'chain_template.npy'),
        'template_run': False,
        'parallelize': True,
    }

    datafile, catalogue = init_setup(
        fit_folder=config['fit_folder'],
        cat_folder=config['cat_folder'],
        data_path=config['data_path'],
        mol_name=config['mol_name'],
        block_interlopers=config['block_interlopers']
    )

    fit_multi_gaussian(
        datafile=datafile,
        fit_folder=config['fit_folder'],
        catalogue=catalogue,
        nruns=config['nruns'],
        mol_name=config['mol_name'],
        prior_path=config['prior_path'],
        restart=config['restart'],
        template_run=config['template_run'],
        parallelize=config['parallelize']
    )

In [ ]:
source_size = 47.07
Ncol        = 6.04E10
Tex         = 7.55
dV          = 0.79
vlsr        = 4.22

datagrid = np.load(datafile, allow_pickle=True)
dsn_freqs, dsn_ints = np.array(datagrid[0]), np.array(datagrid[1])

# Bounds can be identified by hand at this step
print(dsn_freqs)
print(dsn_ints)

In [ ]:
catfile = os.path.join(config['cat_folder'], f"{config['mol_name']}.cat")
mol_cat = MolCat(config['mol_name'], catfile)
freqs, ints, taus = predict_intensities(source_size, Ncol, Tex, dV, mol_cat)
model_ints = make_model(np.array(freqs), np.array(ints), source_size, dsn_freqs, dsn_ints, vlsr, dV, Tex)

# Bounds for HC5N in Cha-C2
bounds = [
    (18638.518485873, 18638.701607659),
    (21301.14895746, 21301.36259954),
    (23963.7778851, 23964.02204748),
    (26626.42084476, 26626.66500714)
]

print(dsn_freqs)
print(model_ints)

fig, axs = plt.subplots(2, 2, figsize=(14, 10))

for i, (lower_bound, upper_bound) in enumerate(bounds):
    mask = (dsn_freqs >= lower_bound) & (dsn_freqs <= upper_bound)
    filtered_dsn_freqs = dsn_freqs[mask]
    filtered_dsn_ints = dsn_ints[mask]
    filtered_model_ints = model_ints[mask]
    
    ax = axs[i // 2, i % 2]
    ax.plot(filtered_dsn_freqs, filtered_dsn_ints, label='DSN Intensities')
    ax.plot(filtered_dsn_freqs, filtered_model_ints, label='Model Intensities')
    ax.scatter(filtered_dsn_freqs, filtered_dsn_ints)
    ax.scatter(filtered_dsn_freqs, filtered_model_ints)
    ax.set_xlim(lower_bound, upper_bound)
    ax.legend()

plt.tight_layout()
plt.show()